In [0]:
from datetime import datetime
import pandas as pd 
from pyspark.sql.functions import current_timestamp

In [0]:
# processName = dbutils.widgets.get("prm_processName")
# nextSourceFileDateSQL = f"""SELECT COALESCE(MAX(PROCESSED_FILE_TABLE_DATE)+1, '2023-01-01') AS NEXT_SOURCE_FILE_DATE FROM pricing_analytics.processrunlogs.deltalakehouse_process_runs
# WHERE PROCESS_NAME = "dailyPricingSourceIngest" AND PROCESS_STATUS = "Completed"
# """
# nextSourceFileDateDF = spark.sql(nextSourceFileDateSQL)
# nextSourceFileDateDF.show()
# nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]

In [0]:
# nextSourceFileDateDF = spark.sql(nextSourceFileDateSQL)
# nextSourceFileDateDF.show()

In [0]:
processName = dbutils.widgets.get("prm_processName")
nextSourceFileDateSQL = f"""SELECT COALESCE(MAX(PROCESSED_FILE_TABLE_DATE)+1, '2023-01-01') AS NEXT_SOURCE_FILE_DATE FROM pricing_analytics.processrunlogs.deltalakehouse_process_runs
WHERE PROCESS_NAME = "dailyPricingSourceIngest" AND PROCESS_STATUS = "Completed"
"""
nextSourceFileDateDF = spark.sql(nextSourceFileDateSQL)
nextSourceFileDateDF.show()
nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]
nextSourceFileDateDF = spark.sql(nextSourceFileDateSQL)
nextSourceFileDateDF.show()
dailyPricingSourceBaseURL = "https://retailpricing.blob.core.windows.net/"
dailyPricingSubFolder = "daily-pricing/"
dailyPricingSourceFileDate = datetime.strptime(str(nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]), "%Y-%m-%d").strftime('%m%d%Y')
dailyPricingSourceFileName = f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"

dailyPricingSinkLayerName = "bronze"
dailyPricingStorageAccountName = "lckudadatalakehousedev"
dailyPricingSinkFolderName = "daily-pricing"

In [0]:
# from datetime import datetime, timedelta
# from pyspark.sql.functions import current_timestamp
# import pandas as pd

# # Initialize variables
# processName = dbutils.widgets.get("prm_processName")
# end_date = datetime.strptime("2023-06-30", "%Y-%m-%d")

# # Get the initial next source file date
# nextSourceFileDateSQL = f"""SELECT COALESCE(MAX(PROCESSED_FILE_TABLE_DATE)+1, '2023-01-01') AS NEXT_SOURCE_FILE_DATE 
#                             FROM pricing_analytics.processrunlogs.deltalakehouse_process_runs
#                             WHERE PROCESS_NAME = 'dailyPricingSourceIngest' AND PROCESS_STATUS = 'Completed'"""
# nextSourceFileDateDF = spark.sql(nextSourceFileDateSQL)
# next_source_file_date = datetime.strptime(
#     str(nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]), 
#     "%Y-%m-%d"
# )

# # Base URL and folder settings
# dailyPricingSourceBaseURL = "https://retailpricing.blob.core.windows.net/"
# dailyPricingSubFolder = "daily-pricing/"
# dailyPricingSinkLayerName = "bronze"
# dailyPricingStorageAccountName = "lckudadatalakehousedev"
# dailyPricingSinkFolderName = "daily-pricing"

# # Loop through dates until end_date (2023-06-30)
# while next_source_file_date <= end_date:
#     dailyPricingSourceFileDate = next_source_file_date.strftime('%m%d%Y')
#     dailyPricingSourceFileName = f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"
#     dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSubFolder + dailyPricingSourceFileName
#     dailyPricingSinkFolderPath = f"abfss://{dailyPricingSinkLayerName}@{dailyPricingStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"

#     try:
#         print(f"Processing file for date: {next_source_file_date.strftime('%Y-%m-%d')}")
#         # Read CSV from source URL
#         dailyPricingPandasDF = pd.read_csv(dailyPricingSourceURL)
#         dailyPricingSparkDF = spark.createDataFrame(dailyPricingPandasDF)
#         dailyPricingSparkDF = dailyPricingSparkDF.withColumn("source_file_load_date", current_timestamp())

#         # Save to bronze layer (CSV)
#         print(f"Found {dailyPricingSourceURL}")
#         print(f"Starting save data to bronze layer")
#         (
#             dailyPricingSparkDF
#             .write
#             .mode("append")
#             .option("header", "true")
#             .csv(dailyPricingSinkFolderPath)
#         )

#         # Insert into bronze table
#         (
#             dailyPricingSparkDF
#             .write
#             .mode("append")
#             .insertInto("pricing_analytics.bronze.daily_pricing")
#         )

#         # Log the process as Completed
#         processStatus = 'Completed'
#         processInsertSql = f"""INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS
#                                (PROCESS_NAME, PROCESSED_FILE_TABLE_DATE, PROCESS_STATUS) 
#                                VALUES('{processName}', '{next_source_file_date.strftime('%Y-%m-%d')}', '{processStatus}')"""
#         spark.sql(processInsertSql)
#         print(f"Successfully processed and logged date: {next_source_file_date.strftime('%Y-%m-%d')}")

#     except Exception as e:
#         print(f"Not Found {dailyPricingSourceURL}")
#         print(f"Error: {str(e)}")
#         # Log the process as Completed even if file not found (as per original logic)
#         processStatus = 'Completed'
#         processInsertSql = f"""INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS
#                                (PROCESS_NAME, PROCESSED_FILE_TABLE_DATE, PROCESS_STATUS) 
#                                VALUES('{processName}', '{next_source_file_date.strftime('%Y-%m-%d')}', '{processStatus}')"""
#         spark.sql(processInsertSql)
#         print(f"Logged date as Completed despite error: {next_source_file_date.strftime('%Y-%m-%d')}")

#     # Increment to the next date
#     next_source_file_date += timedelta(days=1)

# print("Completed processing all dates up to 2023-06-30")

In [0]:

try:
    dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSubFolder + dailyPricingSourceFileName
    dailyPricingPandasDF = pd.read_csv(dailyPricingSourceURL)
    dailyPricingSparkDF = spark.createDataFrame(dailyPricingPandasDF)
    dailyPricingSinkFolderPath = f"abfss://{dailyPricingSinkLayerName}@{dailyPricingStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"
    print(f"Found {dailyPricingSourceURL}")
    print(f"Starting save data to bronze layer")
    dailyPricingSparkDF = dailyPricingSparkDF.withColumn("source_file_load_date", current_timestamp())

    (
        dailyPricingSparkDF.
        write.
        mode("append").
        option("header", "true").
        csv(dailyPricingSinkFolderPath)
    )
    
    (
        dailyPricingSparkDF.
        write.
        mode("append").
        insertInto("pricing_analytics.bronze.daily_pricing")
    )

    processName = dbutils.widgets.get("prm_processName")
    processFileDate = nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]
    processStatus ='Completed'

    processInsertSql = f""" INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_FILE_TABLE_DATE,PROCESS_STATUS) VALUES('{processName}','{processFileDate}','{processStatus}')"""
    spark.sql(processInsertSql)
except Exception as e:
    print(f"Not Found {dailyPricingSourceURL}")
    print(f"Starting Insert Date To Process Table")
    processName = dbutils.widgets.get("prm_processName")
    processFileDate = nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]
    processStatus ='Completed'

    processInsertSql = f""" INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_FILE_TABLE_DATE,PROCESS_STATUS) VALUES('{processName}','{processFileDate}','{processStatus}')"""
    spark.sql(processInsertSql)

# dailyPricingSparkDF = spark.createDataFrame(dailyPricingPandasDF)
# dailyPricingSinkFolderPath = f"abfss://{dailyPricingSinkLayerName}@{dailyPricingStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"

# (
#     dailyPricingSparkDF.
#     withColumn("source_file_load_date", current_timestamp()).
#     write.
#     mode("append").
#     option("header", "true").
#     csv(dailyPricingSinkFolderPath)
# )

# processName = dbutils.widgets.get("prm_processName")
# processFileDate = nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]
# processStatus ='Completed'

# processInsertSql = f""" INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_FILE_TABLE_DATE,PROCESS_STATUS) VALUES('{processName}','{processFileDate}','{processStatus}')"""
# spark.sql(processInsertSql)